In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import importlib
import random
import scipy.optimize as op

In [2]:
# import our own files and reload
import market_data
importlib.reload(market_data)
import capm
importlib.reload(capm)
import portfolio
importlib.reload(portfolio)


<module 'portfolio' from 'c:\\Users\\candy\\Downloads\\Reto_actinver\\quantitative-investment-analysis\\portfolio.py'>

In [3]:
# inputs
notional = 15 # in mn USD
universe = [
    "AA1.MX", "AAL.MX", "AAPL.MX", "AAUN.MX", "ABBV.MX", "ABNB.MX", "AC.MX",
    "ACTINVRB.MX", "AFRM.MX", "AGNC.MX", "ALFAA.MX", "ALPEKA.MX", "ALSEA.MX",
    "AMAT.MX", "AMD.MX", "AMXB.MX", "AMZN.MX", "APA.MX", "ASURB.MX", "ATER.MX",
    "ATOS.MX", "AVGO.MX", "AXP.MX", "BA.MX", "BABAN.MX", "BAC.MX",
    "BBAJIOO.MX", "BIMBOA.MX", "BMY.MX", "BNGO.MX", "BOLSAA.MX",
    "BRKB.MX", "BYND.MX", "C.MX", "CAT.MX", "CCL1N.MX", "CEMEXCPO.MX", 
    "CHDRAUIB.MX", "CLF.MX", "COST.MX", "CRM.MX", "CSCO.MX", 
    "CUERVO.MX", "CVS.MX", "CVX.MX", "DAL.MX", "DIS.MX", "DVN.MX", 
    "ETSY.MX", "F.MX", "FCX.MX", "FDX.MX", "FEMSAUBD.MX",
    "FIBRAMQ12.MX", "FIBRAPL14.MX", "FSLR.MX", "FUBO.MX", "FUNO11.MX", 
    "GAPB.MX", "GCARSOA1.MX", "GCC.MX", "GE.MX", "GENTERA.MX", "GFINBURO.MX", 
    "GFNORTEO.MX", "GILD.MX", "GM.MX", "GME.MX", "GMEXICOB.MX", "GOLDN.MX", 
    "GOOGL.MX", "GRUMAB.MX", "HD.MX", "INTC.MX", "JNJ.MX", "JPM.MX", 
    "KIMBERA.MX", "KO.MX", "KOFUBL.MX", "LABB.MX", "LASITEB-1.MX", "LCID.MX", 
    "LIVEPOLC-1.MX", "LLY.MX", "LUV.MX", "LVS.MX", "LYFT.MX", "MA.MX", 
    "MARA.MX", "MCD.MX", "MEGACPO.MX", "MELIN.MX", "META.MX", "MFRISCOA-1.MX", 
    "MGM.MX", "MRK.MX", "MRNA.MX", "MSFT.MX", "MU.MX", "NCLHN.MX", 
    "NFLX.MX", "NKE.MX", "NKLA.MX", "NUN.MX", "NVAX.MX", "NVDA.MX", "OMAB.MX", 
    "ORBIA.MX", "ORCL.MX", "OXY1.MX", "PARA.MX", "PBRN.MX", "PE&OLES.MX", 
    "PEP.MX", "PFE.MX", "PG.MX", "PINFRA.MX", "PINS.MX", "PLTR.MX", "PYPL.MX", 
    "Q.MX", "QCOM.MX", "RA.MX", "RCL.MX", "RIOT.MX", "RIVN.MX", "ROKU.MX", 
    "SBUX.MX", "SHOPN.MX", "SITES1A-1.MX", "SOFI.MX", "SPCE.MX", 
    "SQ.MX", "T.MX", "TALN.MX", "TERRA13.MX", "TGT.MX",  
    "TMO.MX", "TSLA.MX", "TSMN.MX", "TWLO.MX", "TX.MX", "UAL.MX", "UBER.MX", 
    "UNH.MX", "UPST.MX", "V.MX", "VESTA.MX", "VOLARA.MX", "VZ.MX", "WALMEX.MX", 
    "WFC.MX", "WMT.MX", "WYNN.MX", "X.MX", "XOM.MX", "ZM.MX"
]
rics = universe  # evaluaremos todo el universo

# 1. Cargar y sincronizar las series de retornos de los activos
df_returns = market_data.synchronise_returns(rics)

# 2. Calcular el rendimiento anualizado y la volatilidad de cada activo
def compute_metrics(df_returns, factor=252):
    metrics = []
    for ric in rics:
        # Cálculos de rendimiento anualizado y volatilidad
        annual_return = np.mean(df_returns[ric]) * factor
        volatility = np.std(df_returns[ric]) * np.sqrt(factor)
        sharpe_ratio = annual_return / volatility if volatility != 0 else np.nan
        metrics.append([ric, annual_return, volatility, sharpe_ratio])
    
    # Crear un DataFrame de métricas
    metrics_df = pd.DataFrame(metrics, columns=['Ticker', 'Annual Return', 'Volatility', 'Sharpe Ratio'])
    return metrics_df

# Calcular y mostrar las métricas
metrics_df = compute_metrics(df_returns)
metrics_df.sort_values(by='Annual Return', ascending=False, inplace=True)

# 3. Mostrar los resultados y seleccionar
print('--------------------')
print("DataFrame de métricas de rendimiento y riesgo:")
display(metrics_df)

# Opcional: Filtrar para seleccionar las mejores acciones según el criterio deseado
top_performers = metrics_df.nlargest(10, 'Annual Return')  # Top 10 por rendimiento anual
top_sharpe = metrics_df.nlargest(10, 'Sharpe Ratio')       # Top 10 por ratio de Sharpe
low_volatility = metrics_df.nsmallest(10, 'Volatility')    # Top 10 por menor volatilidad

# Mostrar DataFrames de selección
print("Top 10 por Rendimiento Anual:")
display(top_performers)

print("Top 10 por Ratio de Sharpe:")
display(top_sharpe)

print("Top 10 por Menor Volatilidad:")
display(low_volatility)

c:\Users\candy\Downloads\Reto_actinver\quantitative-investment-analysis\market_data.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ric] = t['return']
c:\Users\candy\Downloads\Reto_actinver\quantitative-investment-analysis\market_data.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ric] = t['return']
c:\Users\candy\Downloads\Reto_actinver\quantitative-investment-analysis\market_data.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

--------------------
DataFrame de métricas de rendimiento y riesgo:


c:\Users\candy\Downloads\Reto_actinver\quantitative-investment-analysis\market_data.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ric] = t['return']
c:\Users\candy\Downloads\Reto_actinver\quantitative-investment-analysis\market_data.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ric] = t['return']
c:\Users\candy\Downloads\Reto_actinver\quantitative-investment-analysis\market_data.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

,Ticker,Annual Return,Volatility,Sharpe Ratio
18,ASURB.MX,5.095929,0.984918,5.173962
20,ATOS.MX,5.051861,0.795635,6.349470
67,GME.MX,4.740587,0.915522,5.178013
125,RIVN.MX,4.216036,0.671262,6.280761
154,X.MX,3.590809,1.003869,3.576968
...,...,...,...,...
56,FUBO.MX,-2.208329,0.657141,-3.360510
104,NVAX.MX,-3.153610,0.563644,-5.595042
19,ATER.MX,-3.592243,0.559308,-6.422650
102,NKLA.MX,-4.290892,1.169054,-3.670398


Top 10 por Rendimiento Anual:


,Ticker,Annual Return,Volatility,Sharpe Ratio
18,ASURB.MX,5.095929,0.984918,5.173962
20,ATOS.MX,5.051861,0.795635,6.349470
67,GME.MX,4.740587,0.915522,5.178013
125,RIVN.MX,4.216036,0.671262,6.280761
154,X.MX,3.590809,1.003869,3.576968
88,MARA.MX,3.443961,1.358036,2.535987
106,OMAB.MX,3.183410,0.625027,5.093236
99,NCLHN.MX,3.098811,0.316331,9.796113
58,GAPB.MX,3.050380,0.718277,4.246804
81,LCID.MX,2.789322,1.051621,2.652402


Top 10 por Ratio de Sharpe:


,Ticker,Annual Return,Volatility,Sharpe Ratio
78,KOFUBL.MX,1.698710,0.171252,9.919358
53,FIBRAMQ12.MX,1.593212,0.161698,9.853038
99,NCLHN.MX,3.098811,0.316331,9.796113
87,MA.MX,0.986183,0.104476,9.439369
36,CEMEXCPO.MX,1.978737,0.216382,9.144638
6,AC.MX,2.117461,0.243582,8.693028
34,CAT.MX,2.467303,0.296197,8.329930
94,MGM.MX,1.880440,0.232039,8.103999
64,GFNORTEO.MX,2.016349,0.262523,7.680648
37,CHDRAUIB.MX,1.214970,0.158232,7.678391


Top 10 por Menor Volatilidad:


,Ticker,Annual Return,Volatility,Sharpe Ratio
7,ACTINVRB.MX,0.358000,0.069009,5.187708
87,MA.MX,0.986183,0.104476,9.439369
74,JNJ.MX,-0.402099,0.115748,-3.473905
41,CSCO.MX,-0.240896,0.136703,-1.762190
115,PG.MX,0.934013,0.146318,6.383448
75,JPM.MX,0.493714,0.148850,3.316863
135,TERRA13.MX,0.661377,0.155461,4.254305
37,CHDRAUIB.MX,1.214970,0.158232,7.678391
15,AMXB.MX,0.610426,0.159479,3.827633
95,MRK.MX,0.263806,0.159995,1.648837
